<a href="https://colab.research.google.com/github/tmandingwa/tmandingwa/blob/main/FFC_Sensitivity_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Sensitivity analysis using a Solver Table

That is, solving the optimization model multiple times as parameters change.

This set of codes guide you on how to perform sensitivity analysis to analyze how changes in model parameters affect your objective function and solution.

In [ ]:
# Usual installation of packages and importing the packages we will use
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pyo
import pandas as pd

## 1.1 Your playground to a step-by-step approach to generate a solver table

In [ ]:
# The usual model setup. For your reference.
model = pyo.ConcreteModel('Cars and Trucks')
model.cars = pyo.Var(domain=pyo.NonNegativeReals)
model.trucks = pyo.Var(domain=pyo.NonNegativeReals)
model.labor = pyo.Constraint(expr=4 * model.cars + 6 * model.trucks <= 500)
model.machine = pyo.Constraint(expr=12 * model.cars + 8 * model.trucks <= 800)
model.profit = pyo.Objective(expr=5 * model.cars + 4 * model.trucks, sense=pyo.maximize)
solver = pyo.SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 380.0, 'Upper bound': 380.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Number of nonzeros': 4, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.010085821151733398}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [ ]:
# TO MODIFY: Steps
# 1. Wrap your codes in a function.
# 2. Update inputs. Labor, Machine.
# 3. Generate outputs. Labor Input, Machine Input, Labor Use, Machine Use, Optimal Cars, Optimal Trucks, Optimal Profit.
def optimize_cars_and_trucks (labor = 500, machine = 800):
  model = pyo.ConcreteModel('Cars and Trucks')
  model.cars = pyo.Var(domain=pyo.NonNegativeReals)
  model.trucks = pyo.Var(domain=pyo.NonNegativeReals)
  model.labor = pyo.Constraint(expr=4 * model.cars + 6 * model.trucks <= labor)
  model.machine = pyo.Constraint(expr=12 * model.cars + 8 * model.trucks <= machine)
  model.profit = pyo.Objective(expr=5 * model.cars + 4 * model.trucks, sense=pyo.maximize)
  solver = pyo.SolverFactory('glpk')
  solver.solve(model)
  return [labor, machine, model.labor(), model.machine(), model.cars(), model.trucks(), model.profit()]

In [ ]:
# Run your codes over a range of labor values
results = [optimize_cars_and_trucks(labor=l) for l in range(500,1000,50)]
results

[[500, 800, 500.0, 800.0, 20.0, 70.0, 380.0],
 [550,
  800,
  550.0,
  799.9999999999999,
  9.99999999999999,
  85.0,
  389.99999999999994],
 [600, 800, 600.0, 800.0, 0.0, 100.0, 400.0],
 [650, 800, 600.0, 800.0, 0.0, 100.0, 400.0],
 [700, 800, 600.0, 800.0, 0.0, 100.0, 400.0],
 [750, 800, 600.0, 800.0, 0.0, 100.0, 400.0],
 [800, 800, 600.0, 800.0, 0.0, 100.0, 400.0],
 [850, 800, 600.0, 800.0, 0.0, 100.0, 400.0],
 [900, 800, 600.0, 800.0, 0.0, 100.0, 400.0],
 [950, 800, 600.0, 800.0, 0.0, 100.0, 400.0]]

In [ ]:
# Display your solutions
df = pd.DataFrame(results, columns=['labor RHS', 'machine RHS', 'labor use', 'machine use', 'cars', 'trucks', 'profit'])
df

,labor RHS,machine RHS,labor use,machine use,cars,trucks,profit
0,500,800,500.0,800.0,20.0,70.0,380.0
1,550,800,550.0,800.0,10.0,85.0,390.0
2,600,800,600.0,800.0,0.0,100.0,400.0
3,650,800,600.0,800.0,0.0,100.0,400.0
4,700,800,600.0,800.0,0.0,100.0,400.0
5,750,800,600.0,800.0,0.0,100.0,400.0
6,800,800,600.0,800.0,0.0,100.0,400.0
7,850,800,600.0,800.0,0.0,100.0,400.0
8,900,800,600.0,800.0,0.0,100.0,400.0
9,950,800,600.0,800.0,0.0,100.0,400.0


## 1.2 Expected output

# 2 Sensitivity analysis using the "model dual".

PYOMO offers an alternative "shorthand" to generate shadow prices. This is very useful if you have a large number of constraints and do not want to calculate / generate shadow prices from the Solver Table.


In [ ]:
# The usual model setup
model = pyo.ConcreteModel('Cars and Trucks')
model.cars = pyo.Var(domain=pyo.NonNegativeReals)
model.trucks = pyo.Var(domain=pyo.NonNegativeReals)
model.labor = pyo.Constraint(expr=4 * model.cars + 6 * model.trucks <= 500)
model.machine = pyo.Constraint(expr=12 * model.cars + 8 * model.trucks <= 800)
model.profit = pyo.Objective(expr=5 * model.cars + 4 * model.trucks, sense=pyo.maximize)

# New line
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Solve
solver = pyo.SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 380.0, 'Upper bound': 380.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Number of nonzeros': 4, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.0046596527099609375}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [ ]:
# Displaying values
model.dual.display()

dual : Direction=IMPORT, Datatype=FLOAT
    Key     : Value
      labor :   0.2
    machine :  0.35


In [ ]:
# Outputting values for later use
model.dual[model.labor]


0.2

In [ ]:
model.dual[model.machine]

0.35